In [24]:
from urllib.request import urlopen
#from datetime import datetime
import time
import json
import pandas as pd
import numpy as np

In [25]:
response = urlopen("https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily")
json_data = response.read().decode('utf-8', 'replace')

raw_json = json.loads(json_data)
df = pd.json_normalize(raw_json)

dates_pr = np.array([])
dates_readable = np.array([])
prices = np.array([])
for entry in df['prices'][0]:
    dates_pr = np.append(dates_pr, entry[0])
    dates_readable = np.append(dates_readable, time.ctime(int(entry[0]) / 1000))
    prices = np.append(prices, entry[1])

mcaps = np.array([])
for entry in df['market_caps'][0]:
    mcaps = np.append(mcaps, entry[1])
    
tv = np.array([])
for entry in df['total_volumes'][0]:
    tv = np.append(tv, entry[1])

data = pd.DataFrame(columns =['date', 'market_caps', 'total_volume', 'price'])
data['date'] = dates_pr
data['price'] = prices
data['market_caps'] = mcaps
data['total_volume'] = tv
data['readable_date'] = dates_readable

In [26]:
# add should_buy column based on next interval price 
buy = np.array([])
for i in range(len(prices)-1):
    if prices[i+1] > prices[i]:
        buy = np.append(buy, 1)
    else:
        buy = np.append(buy, 0)
buy = np.append(buy, 2) # latest row, we don't know next value
data['should_buy'] = buy
    
data

,date,market_caps,total_volume,price,readable_date,should_buy
0,1.367107e+12,1500517590.0,0.000000e+00,135.300000,Sun Apr 28 03:00:00 2013,1.0
1,1.367194e+12,1575032004.0,0.000000e+00,141.960000,Mon Apr 29 03:00:00 2013,0.0
2,1.367280e+12,1501657493.0,0.000000e+00,135.300000,Tue Apr 30 03:00:00 2013,0.0
3,1.367366e+12,1298951550.0,0.000000e+00,117.000000,Wed May 1 03:00:00 2013,0.0
4,1.367453e+12,1148667722.0,0.000000e+00,103.430000,Thu May 2 03:00:00 2013,0.0
...,...,...,...,...,...,...
3366,1.658102e+12,397660610560.549805,2.961579e+10,20824.362752,Mon Jul 18 03:00:00 2022,1.0
3367,1.658189e+12,425756736660.617371,5.863673e+10,22395.365383,Tue Jul 19 03:00:00 2022,1.0
3368,1.658275e+12,445889013251.106384,5.733822e+10,23366.907698,Wed Jul 20 03:00:00 2022,0.0
3369,1.658362e+12,446130012149.910767,4.077572e+10,23313.296732,Thu Jul 21 03:00:00 2022,0.0


In [27]:
from ta.utils import dropna
from ta.volatility import BollingerBands

In [28]:
df = data
# Clean NaN values
#df = dropna(df)

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=df["price"], window=20, window_dev=2)

# Add Bollinger Bands features
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#df = dropna(df)

df

,date,market_caps,total_volume,price,readable_date,should_buy,bb_bbm,bb_bbh,bb_bbl
0,1.367107e+12,1500517590.0,0.000000e+00,135.300000,Sun Apr 28 03:00:00 2013,1.0,NaN,NaN,NaN
1,1.367194e+12,1575032004.0,0.000000e+00,141.960000,Mon Apr 29 03:00:00 2013,0.0,NaN,NaN,NaN
2,1.367280e+12,1501657493.0,0.000000e+00,135.300000,Tue Apr 30 03:00:00 2013,0.0,NaN,NaN,NaN
3,1.367366e+12,1298951550.0,0.000000e+00,117.000000,Wed May 1 03:00:00 2013,0.0,NaN,NaN,NaN
4,1.367453e+12,1148667722.0,0.000000e+00,103.430000,Thu May 2 03:00:00 2013,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3366,1.658102e+12,397660610560.549805,2.961579e+10,20824.362752,Mon Jul 18 03:00:00 2022,1.0,20396.412830,21939.381911,18853.443750
3367,1.658189e+12,425756736660.617371,5.863673e+10,22395.365383,Tue Jul 19 03:00:00 2022,1.0,20502.053975,22271.992491,18732.115460
3368,1.658275e+12,445889013251.106384,5.733822e+10,23366.907698,Wed Jul 20 03:00:00 2022,0.0,20664.972887,22818.346316,18511.599458
3369,1.658362e+12,446130012149.910767,4.077572e+10,23313.296732,Thu Jul 21 03:00:00 2022,0.0,20850.217849,23233.326714,18467.108983


In [29]:
from ta import add_all_ta_features
from ta.utils import dropna

# Clean NaN values
#df = dropna(df)

# Add ta features filling NaN values
df = add_all_ta_features(
    df, open="price", high="price", low="price", close="price", volume="total_volume", fillna=True)


/home/noob/projects/quant/quant/env/lib/python3.9/site-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/noob/projects/quant/quant/env/lib/python3.9/site-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/home/noob/projects/quant/quant/env/lib/python3.9/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/home/noob/projects/quant/quant/env/lib/python3.9/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


In [30]:
df


,date,market_caps,total_volume,price,readable_date,should_buy,bb_bbm,bb_bbh,bb_bbl,volume_adi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,1.367107e+12,1500517590.0,0.000000e+00,135.300000,Sun Apr 28 03:00:00 2013,1.0,NaN,NaN,NaN,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,135.300000,-98.733697,0.000000,0.000000
1,1.367194e+12,1575032004.0,0.000000e+00,141.960000,Mon Apr 29 03:00:00 2013,0.0,NaN,NaN,NaN,0.0,...,0.391243,0.078249,0.312994,0.000000,0.000000,0.000000,137.612475,4.922395,4.805079,4.922395
2,1.367280e+12,1501657493.0,0.000000e+00,135.300000,Tue Apr 30 03:00:00 2013,0.0,NaN,NaN,NaN,0.0,...,0.302152,0.123029,0.179123,0.000000,0.000000,0.000000,136.784400,-4.691462,-4.805079,0.000000
3,1.367366e+12,1298951550.0,0.000000e+00,117.000000,Wed May 1 03:00:00 2013,0.0,NaN,NaN,NaN,0.0,...,-0.855254,-0.072627,-0.782627,0.000000,0.000000,0.000000,129.561906,-13.525499,-14.532060,-13.525499
4,1.367453e+12,1148667722.0,0.000000e+00,103.430000,Thu May 2 03:00:00 2013,0.0,NaN,NaN,NaN,0.0,...,-2.604813,-0.579064,-2.025748,0.000000,0.000000,0.000000,119.690140,-11.598291,-12.327888,-23.555063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3366,1.658102e+12,397660610560.549805,2.961579e+10,20824.362752,Mon Jul 18 03:00:00 2022,1.0,20396.412830,21939.381911,18853.443750,0.0,...,-2.737344,-4.087809,1.350465,8.869451,3.092752,5.776699,20589.264595,-1.739816,-1.755128,15291.251110
3367,1.658189e+12,425756736660.617371,5.863673e+10,22395.365383,Tue Jul 19 03:00:00 2022,1.0,20502.053975,22271.992491,18732.115460,0.0,...,-1.878234,-3.645894,1.767660,14.951795,5.464561,9.487235,20613.516923,7.544061,7.273045,16452.376484
3368,1.658275e+12,445889013251.106384,5.733822e+10,23366.907698,Wed Jul 20 03:00:00 2022,0.0,20664.972887,22818.346316,18511.599458,0.0,...,-0.823180,-3.081351,2.258171,18.507326,8.073114,10.434212,20743.846571,4.338140,4.246679,17170.441758
3369,1.658362e+12,446130012149.910767,4.077572e+10,23313.296732,Thu Jul 21 03:00:00 2022,0.0,20850.217849,23233.326714,18467.108983,0.0,...,-0.017303,-2.468542,2.451239,17.391450,9.936781,7.454669,20972.657721,-0.229431,-0.229695,17130.817984


In [31]:
df = df.drop(columns=['readable_date', 'date', 'market_caps', 'total_volume', 'price'])
df = df.iloc[20: , :] # remove first
df.drop(df.tail(1).index,inplace=True)
# Split data into X & y
X = df.drop("should_buy", axis=1) # use all columns except target
y = df["should_buy"] # we want to predict y using X

df


,should_buy,bb_bbm,bb_bbh,bb_bbl,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
20,0.0,116.096450,136.633819,95.559081,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,...,-2.073085,-3.451824,1.378739,0.000000,0.000000,0.000000,114.744554,0.633631,0.631632,-8.440503
21,1.0,115.023500,131.972660,98.074340,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,...,-1.745858,-3.110630,1.364772,0.000000,0.000000,0.000000,114.989647,-2.727640,-2.765530,-10.937916
22,1.0,114.387500,129.045224,99.729776,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,...,-1.330155,-2.754535,1.424380,0.000000,0.000000,0.000000,115.151028,1.725297,1.710583,-9.401330
23,1.0,114.682500,129.769855,99.595145,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,...,-0.968573,-2.397343,1.428770,0.000000,0.000000,0.000000,115.979157,0.261054,0.260714,-9.164819
24,1.0,115.661000,130.231890,101.090110,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,...,-0.668205,-2.051515,1.383310,0.000000,0.000000,0.000000,116.615626,0.081367,0.081334,-9.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365,0.0,20392.768569,21932.017224,18853.519914,0.0,2.242986e+12,0.0,4.368153e+12,0.0,0.0,...,-3.026864,-4.425426,1.398562,9.988714,1.648577,8.340137,20582.526803,1.915244,1.897134,15563.771872
3366,1.0,20396.412830,21939.381911,18853.443750,0.0,2.213371e+12,0.0,2.184137e+12,-0.0,0.0,...,-2.737344,-4.087809,1.350465,8.869451,3.092752,5.776699,20589.264595,-1.739816,-1.755128,15291.251110
3367,1.0,20502.053975,22271.992491,18732.115460,0.0,2.272007e+12,0.0,1.503190e+13,0.0,0.0,...,-1.878234,-3.645894,1.767660,14.951795,5.464561,9.487235,20613.516923,7.544061,7.273045,16452.376484
3368,0.0,20664.972887,22818.346316,18511.599458,0.0,2.329345e+12,0.0,2.084256e+13,0.0,0.0,...,-0.823180,-3.081351,2.258171,18.507326,8.073114,10.434212,20743.846571,4.338140,4.246679,17170.441758


In [32]:
np.random.seed(42)


In [33]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
# Example use case (requires X & y)
X_train, X_test, y_train, y_test = train_test_split(X, y)




In [34]:
# Random Forest Classifier (for classification problems)
from sklearn.ensemble import RandomForestClassifier
# Instantiating a Random Forest Classifier (clf short for classifier)
clf = RandomForestClassifier()



In [35]:
# All models/estimators have the fit() function built-in
clf.fit(X_train, y_train)



RandomForestClassifier()

In [36]:
# Once fit is called, you can make predictions using predict()
y_preds = clf.predict(X_test)

# You can also predict with probabilities (on classification models)
y_probs = clf.predict_proba(X_test)

In [37]:
# View preds
y_preds, 

(array([0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
        0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
        0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 

In [38]:
# View probabilities
y_probs

array([[0.51, 0.49],
       [0.3 , 0.7 ],
       [0.43, 0.57],
       ...,
       [0.54, 0.46],
       [0.6 , 0.4 ],
       [0.62, 0.38]])

In [39]:
# All models/estimators have a score() function
clf.score(X_test, y_test)


0.5346062052505967